In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
import google.generativeai as genai

In [ ]:
import json
from transformers import AutoTokenizer
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

In [115]:
file_path = "/content/drive/MyDrive/reports-label/Train.jsonl/Train.jsonl"

In [116]:
text1 = []
with open(file_path, "r") as file:
  for i, j in enumerate(file):
    line = json.loads(j)
    text1.append(line)

In [179]:
text1[2699]

{'id': 's55166013',
 'split': 'Train',
 'label': "'No Finding'",
 'text': 'The lungs are well-expanded. The known right upper lobe lung mass is unchanged. The mediastinal contour is stable. Stable mild cardiomegaly. No pneumothorax or pneumomediastinum. No pleural effusion. No acute osseous abnormality.',
 'img': '/home/mimic-cxr/dataset/image_preprocessing/re_512_3ch/Train/s55166013.jpg'}

In [70]:
text[0]

{'id': 's57495790',
 'split': 'Valid',
 'label': "'Lung Opacity'",
 'text': 'Single portable view of the chest. Lower lung volumes seen on the current exam. Patchy region of opacity identified at the left lung base. Elsewhere, the lungs are clear. The cardiomediastinal silhouette is within normal limits. Tortuosity of the descending thoracic aorta is noted. Partially visualized apparently chronic deformity of the proximal right humerus is also seen.',
 'img': '/home/mimic-cxr/dataset/image_preprocessing/re_512_3ch/Valid/s57495790.jpg'}

In [28]:
tokens = word_tokenize(text[0])

In [81]:
print(tokens)

['A', 'tip', 'of', 'a', 'left', 'Port-A-Cath', 'lies', 'in', 'the', 'low', 'superior', 'vena', 'cava', '.', 'Indistinct', 'nodular', 'opacities', 'in', 'the', 'left', 'lung', 'base', 'are', 'consistent', 'with', 'known', 'metastatic', 'disease', 'and', 'better', 'demonstrated', 'on', 'the', 'prior', 'CT', '.', 'There', 'is', 'bilateral', 'pleural', 'thickening', 'and', 'small', 'pleural', 'effusions', 'that', 'appear', 'relatively', 'unchanged', 'in', 'size', 'since', '___', '.', 'The', 'cardiac', 'and', 'mediastinal', 'contours', 'are', 'normal', '.', 'The', 'upper', 'bowel', 'gas', 'pattern', 'is', 'unremarkable', '.']


In [59]:
model = genai.GenerativeModel('gemini-2.0-flash-lite')

In [ ]:
genai.configure(api_key="########")

In [75]:
def create_prompt(text):
  firstline = " Rewrite the following chest X-ray reports so that it does NOT reference any prior studies or comparisons, Do not add your own knowledge to its medical content, and Keep only the parts that are relevant to the current study."
  for i in range(len(text)):
    firstline += f"""\n\n chest X-ray report number {i}: \n{text[i]["text"]}"""

  for j in range(len(text)):
    firstline += f""" \n\n Rewritten report number {i}:"""

  return firstline



In [78]:
new_prompt = create_prompt(text)

In [68]:
prompt

' Rewrite the following chest X-ray report so that it does NOT reference any prior studies or comparisons, Do not add your own knowledge to its medical content, and Keep only the parts that are relevant to the current study.\n\nchest X-ray report: \n{\'id\': \'s55807597\', \'split\': \'Valid\', \'label\': "\'No Finding\'", \'text\': \'Assessment is limited by motion artifact and low lung volumes. Allowing for these limitation, there are no focal parenchymal opacities. Cardiomediastinal and hilar contours are unremarkable with the exception of a tortuous aorta. There is no pleural effusion or pneumothorax.\', \'img\': \'/home/mimic-cxr/dataset/image_preprocessing/re_512_3ch/Valid/s55807597.jpg\'} \n\nRewritten report:'

In [96]:
indicator = [i+1 for i in range(30)]

In [126]:
gen_text = []

for i in indicator:
  if i == 1:
    new_prompt = create_prompt(text1[:90])
  elif i == 30:
    new_prompt = create_prompt(text1[(i-1)*90 :])
  else:
    new_prompt = create_prompt(text1[(i-1)*90 : i*90])

  response = model.generate_content(new_prompt)
  gen_text.append(response.text)


In [168]:
first_2700_gen = []
for i in gen_text:
  repo = re.split(r"\*\*chest X-ray report number \d+:\*\*\n\n", i)
  repo = [repo.replace('\n', '') for repo in repo[1:]]
  if len(repo) == 0:
    repo = re.split(r"\n\nchest X-ray report number \d+:\n", i)
    repo = [repo.replace('\n', '') for repo in repo[1:]]

  if len(repo) == 0:
    repo = re.split(r"\*\*chest X-ray report number \d+:\*\*\n", i)
    repo = [repo.replace('\n', '') for repo in repo[1:]]

  if len(repo) == 0:
    repo = re.split(r"\*\*Report \d+:\*\*\n", i)
    repo = [repo.replace('\n', '') for repo in repo[1:]]

  if len(repo) == 0:
    repo = re.split(r"\*\*Report Number \d+:\*\*\n\n", i)
    repo = [repo.replace('\n', '') for repo in repo[1:]]

  print(len(repo))
  first_2700_gen.extend(repo)

90
90
90
90
87
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90


In [170]:
first_2700_gen

['*   A tip of a left Port-A-Cath lies in the low superior vena cava.*   Indistinct nodular opacities in the left lung base.*   There is bilateral pleural thickening and small pleural effusions.*   The cardiac and mediastinal contours are normal.*   The upper bowel gas pattern is unremarkable.',
 '*   New basal consolidation suggesting pneumonia may be developing.*   Heart size normal.*   Lungs otherwise clear.*   No pleural abnormality.*   ET tube in standard placement.*   Nasogastric tube ends in the stomach.*   No pneumothorax.',
 '*   There is no right pneumothorax or appreciable right pleural effusion.*   There is irregular pleural thickening.*   Left lung is clear and there is no pleural abnormality on that side.*   Heart size normal.',
 '*   A tracheostomy cannula is located within the upper trachea above the level of the clavicles.*   There is no pneumothorax.*   There is mild pulmonary vascular congestion, though no large effusions.*   Cardiomediastinal and hilar contours are 